In [ ]:
import pandas as pd


In [4]:

data = pd.read_csv('df_con_grid.csv')


### Crear variables

In [5]:

# Definir la normalización de las variables


#Consideramos que el riesgo aumenta conforme la temperatura se aleja de un valor seguro, 
# por ejemplo, 30°C. A partir de los 40°C ya el riesgo se vuelve muy alto.
def Temperatura_normalizada(temperature):
    return max(0, min(1, (temperature - 30) / (40 - 30)))

#El riesgo de incendio crece cuando la humedad es muy baja. 
# Consideramos que la humedad es más peligrosa por debajo de un 30%.
def Humedad_normalizada(humidity):
    return max(0, min(1, (30 - humidity) / 30))

#La velocidad del viento también tiene un impacto importante. 
# Por ejemplo, un viento superior a 30 km/h puede hacer que un incendio se propague rápidamente.
def Viento_normalizado(wind_speed):
    return max(0, min(1, (wind_speed - 30) / (60 - 30)))

# Crear la nueva variable 'fire_risk_index' con la fórmula ponderada segun la importancia relativa que he considerado 
# donde he dado menos importancia al la velocidad del viendo que las otras dos variables T y H
def calculate_fire_risk(row):
    Temperature = Temperatura_normalizada(row['Temperature (C)']) 
    Humedad = Humedad_normalizada(row['Humidity (%)']) 
    Viento = Viento_normalizado(row['V_VIENTO']) 
    
    # Índice ponderado 30/30/30
    fire_risk_index = 0.4 * Temperature + 0.4 * Humedad + 0.2 * Viento
    
    return fire_risk_index

# Crear la nueva columna en el DataFrame
data['fire_risk_index'] = data.apply(calculate_fire_risk, axis=1)

# Guardar el DataFrame actualizado en un nuevo archivo CSV
output_file = 'df_con_grid_fire.csv'  #
data.to_csv(output_file, index=False)

print(f"Índice de riesgo de incendio calculado y guardado en '{output_file}'")


Índice de riesgo de incendio calculado y guardado en 'df_con_grid_fire.csv'


In [6]:
# Feature 1: 
data['difficulty_viento_humedad'] = data['DifficultyIndex'] * (data['V_VIENTO'] + data['Humidity (%)'])

# Feature 2:
data['fire_risk_fuel'] = data['fire_risk_index'] * (data['FuelLoadIndex'] / 100)

# Feature 3: 
data['fire_risk_landcover'] = data['fire_risk_index'] * data['LandCover']


output_file = 'df_con_grid_fire.csv'  #
data.to_csv(output_file, index=False)

print(f"Índice de riesgo de incendio calculado y guardado en '{output_file}'")

Índice de riesgo de incendio calculado y guardado en 'df_con_grid_fire.csv'


Creo a continuacion las variables de tipo clase para predecir si hay que madar o no aviones, buldozzer o helicopteros

In [7]:

data = pd.read_csv('df_con_grid_fire.csv')

In [8]:


def AvionAnfibio(range_str):
    if range_str == 0:
        return 0
    
    else:
        return 1


data['AVIANFNUM_CLASS'] = data['AVIANFNUM'].apply(AvionAnfibio)

In [9]:
def AvionTerre(range_str):
    if range_str == 0:
        return 0
    
    else:
        return 1


data['AVICARNUM_CLASS'] = data['AVICARNUM'].apply(AvionTerre)

In [10]:
def map_buldozzer(range_str):
    if range_str == 0:
        return 0
    
    else:
        return 1


data['BULDOZZER_CLASS'] = data['BULDOZZER'].apply(map_buldozzer)

data[['BULDOZZER_CLASS', 'BULDOZZER']].value_counts()

BULDOZZER_CLASS  BULDOZZER
0                0            10471
1                1               34
                 2               12
                 3                5
                 25               2
                 5                1
                 10               1
Name: count, dtype: int64

In [11]:
def map_AUTOBOMBA(range_str):
    if range_str == 0:
        return 0
    
    else:
        return 1


data['AUTOBOMBA_CLASS'] = data['AUTOBOMBA'].apply(map_AUTOBOMBA)